In [1]:
import csv
import pandas as pd
import numpy as np
import ast
from tqdm import tqdm
from requests.compat import urljoin
import requests
import glob
import os
from collections import defaultdict
import re
from bs4 import BeautifulSoup

In [2]:
!python --version

Python 3.10.9


In [3]:
!which python

/home/stirunag/falconframes_env/bin/python


In [4]:
# pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.3/en_core_sci_sm-0.5.3.tar.gz

In [5]:
!python -m spacy validate

✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.7.0) =================
ℹ spaCy installation:
/home/stirunag/falconframes_env/lib/python3.10/site-packages/spacy

NAME             SPACY            VERSION                     
en_core_sci_sm   >=3.7.4,<3.8.0   0.5.4   --> n/a

ℹ The following packages are custom spaCy pipelines or not available
for spaCy v3.7.0:
en_core_sci_sm


In [6]:
import spacy
# nlp = spacy.load("en_core_sci_sm")
nlp = spacy.load("en_core_sci_sm")

/home/stirunag/falconframes_env/lib/python3.10/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_sci_sm' (0.5.4) was trained with spaCy v3.7.4 and may not be 100% compatible with the current version (3.7.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/home/stirunag/falconframes_env/lib/python3.10/site-packages/spacy/language.py:2170: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


In [7]:
import pandas as pd
import ast
import numpy as np

# Function to safely evaluate string as a list, returning an empty list if the evaluation fails
def safe_literal_eval(val):
    try:
        return ast.literal_eval(val)
    except (ValueError, SyntaxError):
        return []

# Function to ensure proper string handling
def safe_string(val):
    if pd.isna(val):
        return ""
    return str(val)

# Function to load CSV file and convert 'ner_ines' to a list
def load_csv_with_list_column(file_path):
    df = pd.read_csv(file_path, usecols=['sentence', 'ner_ines'])
    # Replace NaN in 'ner_ines' and 'sentence' columns
    df['ner_ines'] = df['ner_ines'].replace(np.nan, '[]').apply(safe_literal_eval)
    df['sentence'] = df['sentence'].apply(safe_string)
    return df

# Load your files
train_df = load_csv_with_list_column('../data/DICT2ML/D2M_train.csv')
dev_df = load_csv_with_list_column('../data/DICT2ML/D2M_dev.csv')
test_df = load_csv_with_list_column('../data/DICT2ML/D2M_test.csv')



In [8]:
train_df

,sentence,ner_ines
0,GR3027 antagonizes GABAA receptor-potentiating...,"[[135, 149, hyperammonemia, DS], [117, 121, ra..."
1,M.,[]
2,Johansson and A.,[]
3,Agusti contributed equally to this work.,[]
4,Hepatic encephalopathy (HE) is one of the prim...,"[[67, 82, liver cirrhosis, DS], [0, 22, Hepati..."
...,...,...
80020,ABSTRACT: Coronavirus disease 2019 (COVID-19) ...,"[[10, 34, Coronavirus disease 2019, DS], [36, ..."
80021,Bilateral ground-glass or patchy opacity (89.6...,[]
80022,RESULTS: An approximately 1:1 ratio of male (5...,"[[63, 71, COVID-19, DS]]"
80023,"ABSTRACT: Since late December 2019, an outbrea...","[[51, 82, 2019 novel coronavirus diseases, DS]..."


In [9]:
# Define the exact match list for 'GO' entities to be removed (all converted to lowercase)
go_entities_to_remove = {
    "uptake", "transport", "intracellular", "fusion", "behaviors", "complex",
    "behavior", "behavior'", "behaviors", "behavior”", "behaviour", "behaviours",
    "gems", "learning", "memory", "protein bodies", "protein body", "regeneration",
    "swimming", "translation", "transports", "tooth replacement", "system development",
    "taste", "smoother", "sex determination", "sex differentiation", "pores"
}

# Function to clean 'GO' entities from the ner_ines list if they match the removal list (case-insensitive)
def clean_go_entities(dataframes):
    for df in dataframes:
        for index, row in df.iterrows():
            cleaned_ner_ines = []
            ner_ines = row['ner_ines']
            for entity in ner_ines:
                if len(entity) == 4:
                    _, _, text, entity_type = entity
                    # Remove entity if it is 'GO' and its text (lowercased) matches any in the removal list
                    if entity_type == "GO" and text.lower() in go_entities_to_remove:
                        continue
                cleaned_ner_ines.append(entity)
            # Update the ner_ines column with the cleaned list
            df.at[index, 'ner_ines'] = cleaned_ner_ines

# Clean 'GO' entities from train, dev, and test datasets
clean_go_entities([train_df, dev_df, test_df])


In [10]:
import re

# Define the exact match list for 'EM' entities to be removed (all converted to lowercase)
em_entities_to_remove = {
    "insertion", "recombination", "co-expression", "recombination'", "co-expression", "translocations",
    "homologous recombination", "acetylation/deacetylation", "maf", "itc", "sga", "iss", "chips",
    "inversion", "deacetylation", "protein folding", "ubiquination", "acetylation", "translocation",
    "polymerization", "sedation", "three-hybrid system", "transfection", "transfections", "transgenic",
    "translocation", "translocations", "two-hybrid", "urine collection", "scraping", "insertion", 
    "insertions", "inversion", "inversions"
}

# Define the base terms to look for in phrases that include "assay"
base_terms_to_check = [
    "luminescent prey", "rt-pcr", "thermal stability", 
    "electrophoretic mobility shift", "cold tolerance", "microdilution"
]

# Function to clean 'EM' entities from the ner_ines list if they match the removal list (case-insensitive)
def clean_em_entities_and_annotate(dataframes):
    for df in dataframes:
        for index, row in df.iterrows():
            cleaned_ner_ines = []
            ner_ines = row['ner_ines']
            original_sentence = row['sentence']
            sentence_lower = original_sentence.lower()  # Convert sentence to lowercase for consistent matching

            # Clean existing EM entities
            for entity in ner_ines:
                if len(entity) == 4:
                    _, _, text, entity_type = entity
                    # Remove entity if it is 'EM' and its text (lowercased) matches any in the removal list
                    if entity_type == "EM" and text.lower() in em_entities_to_remove:
                        continue
                cleaned_ner_ines.append(entity)

            # Look for "assay" and check preceding words
            words = sentence_lower.split()
            for i, word in enumerate(words):
                if "assay" in word:  # Identify the word containing "assay"
                    # Check up to 3 words before to find a match with our terms
                    for n in range(1, 4):
                        if i - n < 0:
                            break
                        preceding_words = " ".join(words[i - n:i])
                        # Match if preceding words form a known base term
                        if preceding_words in base_terms_to_check:
                            start_index = sentence_lower.find(preceding_words)
                            end_index = sentence_lower.find(word, start_index) + len(word)
                            full_term = original_sentence[start_index:end_index]
                            print(f"Detected EM term '{full_term}' in sentence: '{original_sentence}'")

                            # Always add the new entity, avoid unnecessary merge checks
                            cleaned_ner_ines.append([start_index, end_index, full_term, "EM"])
                            # break  # Stop after adding to avoid double-counting
            
            # Update the ner_ines column with the cleaned and potentially extended list
            df.at[index, 'ner_ines'] = cleaned_ner_ines

# Apply the cleaning and annotating function to train, dev, and test datasets
clean_em_entities_and_annotate([train_df, dev_df, test_df])


Detected EM term 'thermal stability assay' in sentence: 'A second binding assay was employed based upon MBP-E6 unfolding in the Sypro Orange- based thermal stability assay (TSA) [25, 33]. '
Detected EM term 'thermal stability assay' in sentence: 'The thermal stability assay [25, 33] was performed with the following modifications. '
Detected EM term 'thermal stability assay.' in sentence: 'Summary of the apparent Kd and Bmax values of CAF-25, -26, -27 and CAF-40 with MBP-E6 wild-type and mutant proteins determined by thermal stability assay. '
Detected EM term 'Electrophoretic mobility shift assay' in sentence: 'Electrophoretic mobility shift assay (EMSA) of RNA'
Detected EM term 'Cold tolerance assays' in sentence: 'Cold tolerance assays'
Detected EM term 'cold tolerance assay.' in sentence: 'S3), were selected for cold tolerance assay. '
Detected EM term 'microdilution assay' in sentence: 'In brief, a microdilution assay in flat bottomed 96-well microtiter plates (TPP, Trasadingen, Sw

In [11]:
# Function to collect entity texts into a dictionary
def collect_entity_texts(*dataframes):
    entity_dict = defaultdict(set)
    for df in dataframes:
        for _, row in df.iterrows():
            ner_ines = row['ner_ines']
            for entity in ner_ines:
                if len(entity) == 4:
                    start, end, text, entity_type = entity
                    entity_dict[entity_type].add(text)
    return dict(entity_dict)

# Collecting entities from all three datasets
entity_texts_dict = collect_entity_texts(train_df, dev_df, test_df)


In [12]:
'microdilution assay' in entity_texts_dict['EM']

True

In [13]:
train_df

,sentence,ner_ines
0,GR3027 antagonizes GABAA receptor-potentiating...,"[[135, 149, hyperammonemia, DS], [117, 121, ra..."
1,M.,[]
2,Johansson and A.,[]
3,Agusti contributed equally to this work.,[]
4,Hepatic encephalopathy (HE) is one of the prim...,"[[67, 82, liver cirrhosis, DS], [0, 22, Hepati..."
...,...,...
80020,ABSTRACT: Coronavirus disease 2019 (COVID-19) ...,"[[10, 34, Coronavirus disease 2019, DS], [36, ..."
80021,Bilateral ground-glass or patchy opacity (89.6...,[]
80022,RESULTS: An approximately 1:1 ratio of male (5...,"[[63, 71, COVID-19, DS]]"
80023,"ABSTRACT: Since late December 2019, an outbrea...","[[51, 82, 2019 novel coronavirus diseases, DS]..."


In [19]:
import os
import csv
import pathlib
from tqdm import tqdm
import nltk
from nltk.tokenize import wordpunct_tokenize

# Ensure NLTK is installed and the tokenizer is available
nltk.download('punkt')

def find_sub_span(token_span, entity_span):
    if token_span[0] < entity_span[1] and token_span[1] > entity_span[0]:
        return max(token_span[0], entity_span[0]), min(token_span[1], entity_span[1])
    return None

def convert_to_iob(texts, ner_tags_list):
    results = []

    for text, ner_tags in zip(texts, ner_tags_list):
        # Tokenize using NLTK's wordpunct_tokenizer
        tokens = wordpunct_tokenize(text)
        token_spans = []
        current_idx = 0

        # Calculate token spans based on the original text
        for token in tokens:
            start_idx = text.find(token, current_idx)
            end_idx = start_idx + len(token)
            token_spans.append((start_idx, end_idx))
            current_idx = end_idx

        iob_tags = ['O'] * len(tokens)

        for start, end, entity, entity_type in sorted(ner_tags, key=lambda x: x[0]):
            entity_flag = False  # Flag to indicate if we are inside an entity
            for i, token_span in enumerate(token_spans):
                if find_sub_span(token_span, (start, end)):
                    if not entity_flag:  # If it's the start of an entity
                        iob_tags[i] = 'B-' + entity_type
                        entity_flag = True
                    elif iob_tags[i] == 'O':  # Continue tagging inside of the entity
                        iob_tags[i] = 'I-' + entity_type
                else:
                    entity_flag = False  # Reset flag when we're no longer in an entity

        results.append(list(zip(tokens, iob_tags)))
    return results

def convert_to_IOB_format_from_df(dataframe, output_folder, filename, batch_size=500):
    # Prepare data for batch processing
    data = [(row['sentence'], row['ner_ines']) for index, row in dataframe.iterrows()]

    pathlib.Path(output_folder).mkdir(parents=True, exist_ok=True)
    result_path = os.path.join(output_folder, filename)

    with open(result_path, 'w', newline='\n') as f1:
        train_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')

        for i in tqdm(range(0, len(data), batch_size), desc="Processing batches"):
            batch = data[i:i+batch_size]
            sentences, ner_tags_batch = zip(*batch)

            # Convert to IOB format
            batch_results = convert_to_iob(sentences, ner_tags_batch)

            for tagged_tokens in batch_results:
                for each_token in tagged_tokens:
                    train_writer.writerow(list(each_token))
                train_writer.writerow('')

# Example usage:
output_folder = '../data/DICT2ML/'

# Convert train, dev, and test dataframes to IOB format
convert_to_IOB_format_from_df(train_df, output_folder, 'train_IOB.tsv')
convert_to_IOB_format_from_df(dev_df, output_folder, 'dev_IOB.tsv')
convert_to_IOB_format_from_df(test_df, output_folder, 'test_IOB.tsv')


[nltk_data] Downloading package punkt to /home/stirunag/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Processing batches: 100%|████████████████████████████████████████████████████████████| 36/36 [00:00<00:00, 130.19it/s]


In [20]:
import pandas as pd

def load_iob_file(file_path):
    """
    Function to load an IOB file into a pandas DataFrame.
    
    Parameters:
    file_path (str): Path to the IOB file.
    
    Returns:
    pd.DataFrame: DataFrame containing tokens and their IOB tags.
    """
    data = []
    with open(file_path, 'r') as f:
        for line_num, line in enumerate(f, 1):
            line = line.strip()
            if line:  # Only process non-empty lines
                parts = line.split("\t")
                if len(parts) == 2:
                    token, tag = parts
                    data.append((token, tag))
                # else:
                    # print(f"Warning: Skipping invalid line {line_num} in file {file_path}: '{line}'")
            else:
                data.append(("", ""))  # Add empty line as a sentence boundary
    return pd.DataFrame(data, columns=["token", "tag"])


def check_data_integrity(df):
    """
    Function to check the integrity of NER data.
    Ensures:
    1. Each 'B-ent' is followed by its respective 'I-ent' (if applicable).
    2. No 'I-ent' appears without a preceding 'B-ent'.
    
    Parameters:
    df (pd.DataFrame): DataFrame containing the NER columns with tokens and entities.
    
    Returns:
    bool: True if the data integrity is intact, False if there are issues.
    issues (list): List of issues found in the dataset.
    """
    issues = []
    found_b_ent = False

    for i, entity in enumerate(df['tag']):
        if entity.startswith('B-'):
            found_b_ent = True
        elif entity.startswith('I-'):
            if not found_b_ent:
                issues.append(f"Row: {i}, Hanging I-ent without preceding B-ent.")
            # Check if the current I-ent is the correct continuation of a B-ent
            elif not entity[2:] == df['tag'].iloc[i-1][2:]:
                issues.append(f"Row: {i}, Mismatch between B-ent and I-ent.")
        # Reset B-ent flag if it's an 'O', empty string, or sentence boundary
        if entity == 'O' or entity == '' or (df['token'].iloc[i] == "" and entity == ""):
            found_b_ent = False

    return (True, []) if not issues else (False, issues)

def check_integrity_of_files(train_file_paths, dev_file_paths, test_file_paths):
    """
    Check the integrity of NER train, development, and test sets for each dataset.
    
    Parameters:
    train_file_paths (list): List of file paths for training datasets.
    dev_file_paths (list): List of file paths for development datasets.
    test_file_paths (list): List of file paths for test datasets.
    
    Returns:
    None: Prints the integrity check results for each set.
    """
    for i, (train_file, dev_file, test_file) in enumerate(zip(train_file_paths, dev_file_paths, test_file_paths)):
        print(f"\nChecking Dataset {i+1}:")

        # Load train set and check integrity
        train_df = load_iob_file(train_file)
        is_train_valid, train_issues = check_data_integrity(train_df)
        if is_train_valid:
            print(f"Train set {i+1} is valid.")
        else:
            print(f"Train set {i+1} has issues:\n", "\n".join(train_issues))
        
        # Load dev set and check integrity
        dev_df = load_iob_file(dev_file)
        is_dev_valid, dev_issues = check_data_integrity(dev_df)
        if is_dev_valid:
            print(f"Dev set {i+1} is valid.")
        else:
            print(f"Dev set {i+1} has issues:\n", "\n".join(dev_issues))

        # Load test set and check integrity
        test_df = load_iob_file(test_file)
        is_test_valid, test_issues = check_data_integrity(test_df)
        if is_test_valid:
            print(f"Test set {i+1} is valid.")
        else:
            print(f"Test set {i+1} has issues:\n", "\n".join(test_issues))

# Example usage:
train_files = ['../data/DICT2ML/train_IOB.tsv']
dev_files = ['../data/DICT2ML/dev_IOB.tsv']
test_files = ['../data/DICT2ML/test_IOB.tsv']

check_integrity_of_files(train_files, dev_files, test_files)



Checking Dataset 1:
Train set 1 is valid.
Dev set 1 is valid.
Test set 1 is valid.


In [21]:
# Function to combine train and dev dataframes, then convert to IOB format
def save_combined_to_iob(train_df, dev_df, test_df, output_folder):
    # Combine train and dev dataframes
    train_dev_df = pd.concat([train_df, dev_df], ignore_index=True)
    
    # Save combined train_dev as train_dev.tsv
    convert_to_IOB_format_from_df(train_dev_df, output_folder, 'train_dev.tsv')
    
    # Save test as dev.tsv
    convert_to_IOB_format_from_df(test_df, output_folder, 'dev.tsv')

# Example usage:
output_folder = '../data/DICT2ML/'

# Assuming train_df, dev_df, and test_df are already loaded
save_combined_to_iob(train_df, dev_df, test_df, output_folder)


Processing batches: 100%|█████████████████████████████████████████████████████████████| 36/36 [00:00<00:00, 97.92it/s]


In [22]:
# Example usage:
train_files = ['../data/DICT2ML/train_dev.tsv']
dev_files = ['../data/DICT2ML/dev.tsv']
check_integrity_of_files(train_files, dev_files, test_files)


Checking Dataset 1:
Train set 1 is valid.
Dev set 1 is valid.
Test set 1 is valid.
